# About: VM - Go! with prepared VM image

---

Start VM instance with prepared VM image using livbirt.

libvirtがインストールされている仮想化基盤上で、VMを起動するためのNotebook。

すでに**VMイメージ作成Notebook**により、イメージが作成されているものとする。

## *Operation Note*

*This is a cell for your own recording.  ここに経緯を記述*

# Notebookと環境のBinding

Inventory中のgroup名でBind対象を指示する。

In [1]:
target_group = 'test-hadoop-hypervisor'

Bind対象への疎通状態を確認する。

In [2]:
!ansible -m ping {target_group}

XXX.XXX.XXX.106 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


対象マシンにlibvirtがインストールされているかを確認する。

In [3]:
!ansible -b -a 'virsh version' {target_group}

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
Compiled against library: libvirt 0.10.2
Using library: libvirt 0.10.2
Using API: QEMU 0.10.2
Running hypervisor: QEMU 0.12.1



# VMイメージの指定

作成対象のVMのあるディレクトリを指定する。**VMイメージ作成Notebook**により生成されたイメージが格納されているディレクトリを指定すること。

In [4]:
image_base_dir = '/mnt/centos6-base-vm'

以下の2つのファイルが存在している必要がある。

- base.img
- libvirt-base.xml

In [5]:
!ansible -b -a "ls -la {image_base_dir}" {target_group}

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
total 2722840
drwxr-xr-x 2 root root         4096 Jul 29 16:53 .
drwxr-xr-x 5 root root         4096 Aug 18 19:04 ..
-rwxr-xr-x 1 root root 107374182400 Jul 29 16:53 base.img
-rw-r--r-- 1 root root         2839 Jul 29 16:53 libvirt-base.xml



作成するVM名のリストを指定する。お手本では例として2つのVMを指定している。

起動したいVM名をlistで指定すること。**既存のVMと重複してはならない。**

In [6]:
vm_names = map(lambda index: 'test-vm-{}'.format(index + 1), range(0, 7))
vm_names

['test-vm-1',
 'test-vm-2',
 'test-vm-3',
 'test-vm-4',
 'test-vm-5',
 'test-vm-6',
 'test-vm-7']

# VMの作成

VM用のファイルは以下のように作成される。

- /mnt
  - (VM名).xml ... libvirtに与えるXML定義
  - (VM名).img ... VM用の仮想ディスク

## XML定義の生成

基本となるXML定義を得る。

In [7]:
import tempfile
work_dir = tempfile.mkdtemp()
work_dir

'/tmp/tmpNnS7H9'

In [8]:
!ansible -b -m fetch -a 'src={image_base_dir}/libvirt-base.xml dest={work_dir}/libvirt-base.xml flat=yes' {target_group}

XXX.XXX.XXX.106 | SUCCESS => {
    "changed": true, 
    "checksum": "d42215623ad5fca3424317f7d2623fb48b09fdd7", 
    "dest": "/tmp/tmpNnS7H9/libvirt-base.xml", 
    "md5sum": "f34e277e56387fadb8144b5b67139949", 
    "remote_checksum": "d42215623ad5fca3424317f7d2623fb48b09fdd7", 
    "remote_md5sum": null
}


基本のXML定義に基づいて、VM用定義を生成する。

In [9]:
import xml.etree.ElementTree as ET
import virtinst.util
import os

for n in vm_names:
    vmxml = ET.parse(os.path.join(work_dir, 'libvirt-base.xml')).getroot()
    vmxml.find('name').text = n
    vmxml.find('memory').text = '10485760'
    vmxml.find('currentMemory').text = '10485760'
    vmxml.find('vcpu').text = '2'
    vmxml.find('devices').find('disk').find('source').attrib['file'] = os.path.join('/mnt', n + '.img')
    vmxml.find('devices').find('interface').find('mac').attrib['address'] = virtinst.util.randomMAC()
    ET.ElementTree(vmxml).write(os.path.join(work_dir, n + '.xml'))
!ls -la {work_dir}/*.xml

-rw-r--r-- 1 root root 2839 Aug 26 11:04 /tmp/tmpNnS7H9/libvirt-base.xml
-rw-r--r-- 1 root root 2890 Aug 26 11:04 /tmp/tmpNnS7H9/test-vm-1.xml
-rw-r--r-- 1 root root 2890 Aug 26 11:04 /tmp/tmpNnS7H9/test-vm-2.xml
-rw-r--r-- 1 root root 2890 Aug 26 11:04 /tmp/tmpNnS7H9/test-vm-3.xml
-rw-r--r-- 1 root root 2890 Aug 26 11:04 /tmp/tmpNnS7H9/test-vm-4.xml
-rw-r--r-- 1 root root 2890 Aug 26 11:04 /tmp/tmpNnS7H9/test-vm-5.xml
-rw-r--r-- 1 root root 2890 Aug 26 11:04 /tmp/tmpNnS7H9/test-vm-6.xml
-rw-r--r-- 1 root root 2890 Aug 26 11:04 /tmp/tmpNnS7H9/test-vm-7.xml


ホストに定義ファイルをコピーする。

In [10]:
for n in vm_names:
    !ansible -b -m copy -a 'src={work_dir}/{n}.xml dest=/mnt/{n}.xml' {target_group}

XXX.XXX.XXX.106 | SUCCESS => {
    "changed": true, 
    "checksum": "7f6357a82b62cc160faa7da3299023aa6ce3c26f", 
    "dest": "/mnt/test-vm-1.xml", 
    "gid": 0, 
    "group": "root", 
    "md5sum": "4b2f28591273396d8dcc23bce8816a9f", 
    "mode": "0644", 
    "owner": "root", 
    "size": 2890, 
    "src": "/home/ansible/.ansible/tmp/ansible-tmp-1472177063.81-253383976698398/source", 
    "state": "file", 
    "uid": 0
}
XXX.XXX.XXX.106 | SUCCESS => {
    "changed": true, 
    "checksum": "bfdf8615833a0ed6e3f1ac3581085134123bd3fa", 
    "dest": "/mnt/test-vm-2.xml", 
    "gid": 0, 
    "group": "root", 
    "md5sum": "d2677b2ab5378254e10939367206b750", 
    "mode": "0644", 
    "owner": "root", 
    "size": 2890, 
    "src": "/home/ansible/.ansible/tmp/ansible-tmp-1472177066.13-25911206026684/source", 
    "state": "file", 
    "uid": 0
}
XXX.XXX.XXX.106 | SUCCESS => {
    "changed": true, 
    "checksum": "3e772ed9ba7bd561b70fde794694cd13d1f3d862", 
    "dest": "/mnt/test-vm-3.xml",

## イメージファイルのコピー

イメージファイルをVM用に複製する。

In [11]:
for n in vm_names:
    !ansible -b -a 'cp {image_base_dir}/base.img /mnt/{n}.img' {target_group}

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>


XXX.XXX.XXX.106 | SUCCESS | rc=0 >>


XXX.XXX.XXX.106 | SUCCESS | rc=0 >>


XXX.XXX.XXX.106 | SUCCESS | rc=0 >>


XXX.XXX.XXX.106 | SUCCESS | rc=0 >>


XXX.XXX.XXX.106 | SUCCESS | rc=0 >>


XXX.XXX.XXX.106 | SUCCESS | rc=0 >>




## VMの起動

XMLファイル、仮想ディスクファイルがあるかどうかを確認する。

In [12]:
for n in vm_names:
    !ansible -a 'ls -la /mnt/{n}.img /mnt/{n}.xml' {target_group}

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
-rwxr-xr-x 1 root root 107374182400 Aug 26 11:05 /mnt/test-vm-1.img
-rw-r--r-- 1 root root         2890 Aug 26 11:04 /mnt/test-vm-1.xml

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
-rwxr-xr-x 1 root root 107374182400 Aug 26 11:06 /mnt/test-vm-2.img
-rw-r--r-- 1 root root         2890 Aug 26 11:04 /mnt/test-vm-2.xml

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
-rwxr-xr-x 1 root root 107374182400 Aug 26 11:08 /mnt/test-vm-3.img
-rw-r--r-- 1 root root         2890 Aug 26 11:04 /mnt/test-vm-3.xml

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
-rwxr-xr-x 1 root root 107374182400 Aug 26 11:09 /mnt/test-vm-4.img
-rw-r--r-- 1 root root         2890 Aug 26 11:04 /mnt/test-vm-4.xml

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
-rwxr-xr-x 1 root root 107374182400 Aug 26 11:10 /mnt/test-vm-5.img
-rw-r--r-- 1 root root         2890 Aug 26 11:04 /mnt/test-vm-5.xml

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
-rwxr-xr-x 1 root root 107374182400 Aug 26 11:11 /mnt/test-vm-6.img
-rw-r--r-- 1 root root         

VMを起動する。

In [13]:
import time
for n in vm_names:
    !ansible -b -a 'virsh create /mnt/{n}.xml' {target_group}
    time.sleep(60)

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
Domain test-vm-1 created from /mnt/test-vm-1.xml

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
Domain test-vm-2 created from /mnt/test-vm-2.xml

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
Domain test-vm-3 created from /mnt/test-vm-3.xml

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
Domain test-vm-4 created from /mnt/test-vm-4.xml

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
Domain test-vm-5 created from /mnt/test-vm-5.xml

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
Domain test-vm-6 created from /mnt/test-vm-6.xml

XXX.XXX.XXX.106 | SUCCESS | rc=0 >>
Domain test-vm-7 created from /mnt/test-vm-7.xml



VMに設定されたIPアドレスを確認する。

In [14]:
import re
def get_mac_address(vmname):
    domiflist_stdio = !ansible -b -a "virsh domiflist {vmname}" {target_group}
    mac_pattern = re.compile(r'.*bridge.*\s([0-9a-f\:]+)\s*')
    vmmac = [mac_pattern.match(line).group(1) for line in domiflist_stdio if mac_pattern.match(line)][0]
    return vmmac

def get_ip_address(vmmac):
    leases_stdio = !ansible -b -a "grep {vmmac} /var/lib/dnsmasq/dnsmasq.leases" {target_group}
    ip_pattern = re.compile(r'.*\s([0-9a-f\:]+)\s+([0-9\.]+)\s.*')
    ipaddr = [ip_pattern.match(line).group(2) for line in leases_stdio if ip_pattern.match(line)][0]
    return ipaddr

In [15]:
vmdescs = zip(vm_names, map(lambda mac: get_ip_address(mac), map(lambda n: get_mac_address(n), vm_names)))
vmdescs

[('test-vm-1', 'XXX.XXX.XXX.112'),
 ('test-vm-2', 'XXX.XXX.XXX.113'),
 ('test-vm-3', 'XXX.XXX.XXX.114'),
 ('test-vm-4', 'XXX.XXX.XXX.73'),
 ('test-vm-5', 'XXX.XXX.XXX.70'),
 ('test-vm-6', 'XXX.XXX.XXX.71'),
 ('test-vm-7', 'XXX.XXX.XXX.72')]

# Inventoryの更新

Inventoryに、作成したマシンのIPアドレスを追加する。変更する前に、現在の内容をコピーしておく。

In [16]:
!cp inventory {work_dir}/inventory-old

[Inventory](../edit/inventory) を修正する。

In [17]:
!diff -ur {work_dir}/inventory-old inventory

--- /tmp/tmpNnS7H9/inventory-old	2016-08-26 11:21:29.075212671 +0900
+++ inventory	2016-08-26 11:22:24.895689120 +0900
@@ -1,6 +1,15 @@
 [test-hadoop-hypervisor]
 XXX.XXX.XXX.106
 
+[test-hadoop-vm]
+XXX.XXX.XXX.70
+XXX.XXX.XXX.71
+XXX.XXX.XXX.72
+XXX.XXX.XXX.73
+XXX.XXX.XXX.112
+XXX.XXX.XXX.113
+XXX.XXX.XXX.114
+
 [docker_host]
 XXX.XXX.XXX.195
 XXX.XXX.XXX.196


追加したグループ名でpingが通じるかどうかを確認する。

In [18]:
target_vmgroup = 'test-hadoop-vm'

In [19]:
!ansible -m ping {target_vmgroup}

XXX.XXX.XXX.70 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.72 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.71 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.112 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.73 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.113 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}
XXX.XXX.XXX.114 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


# 後始末

一時ディレクトリを削除する。

In [20]:
!rm -fr {work_dir}